# Karl Task Solver using Policy Gradient Algorithms
This projects aims at solving any generic 4x4 Karel task using policy gradient algorithms in an optimal manner (least sequence of actions length). 
# Classes
## Karel(gym.Env):
This is the main class which represents the Karel task as a gym environemnt. Karel inherits from `gym.Env`.
### Constructor Input:
1. `paths` list(str): a list of string paths for the Karel tasks in the dataset.
2. `state_space` (str): default "S0": can have values "S1" and "S2" to change the binary state representation.
3.  `conssequential_loading`: choose whether to load tasks sequentially as ordered in their directory or randomly sampled.
### Important Functions 
1. `reset(Self):` increpement the `current_ep` index and returns the new task the state is pointing at. Should be called at the beginning once after the initalization. 
2. `reward(self, human_state, action):` has the following format:
```
        if action == Action.finish.value:
            if cur_grid == post_grid:
                self.solved = True
                return 1
        elif self.CRASH:
            return -1
        return 0
```
2. `step(self, action):` takes action returns: next state, reward, done (boolean), info (dict showing information)
3. `render(self, mode='console'):` prints the current environemnt (state). Supports `console` option. 
## PolicyGradientBase
This is the base class for all policy gradients algorithms. It abstract non-algorithm specific functionality that are common between them. Whenever this parent class is classes by its two child classes (SAC and A2C), the following parameters are to be passed:

1. env: a gym environemnt object
2. network: MLPA2C or MLPSAC network (ideally can take any PyTorch network with certain specifications.
3. gamma (default: 0.99): the discount factor
4. lr (default: 03e-4): learning rate
5. evaluate_every_n: (default: 500): evaluate after iterating over n episodes
6. show_every_n (default:100): rendering option during evaluation. # not recommended for speed
7. alpha=0.5: step size for actor and critic updates
8. update_every_n =5: update after storing n episodes in the replay_buffer
9. show_every_n (default:100): rendering option during evaluation. # not
10. epochs = 5: the number of rounds over the entire dataset 
11. num_actions=6: action diemnsion 
12. learn_by_demo=True: set to learn by demonstration (i.e, get actions from expert dataset)
13. load_model=False: set to load pre-trained model (Expected ot be in `models/dir`
### Functions (major ones)
1. `train(self, dataset_dict, dataset_all, expert_trajectories=None):` trains the policy passed to the class during construction using the provided dataset, evaluates it. Uses expert_trajectories if `learn_by_demo` is set.
2. `eval_model(self, ds):` takes a dataset and evaluates the model on it. Returns a dict of evaluation stats.
3. `load_algo_policy(self):`: loads the state dictionary for the policy. Should be in 'stats' folder with the same name of the model.
4. `gen_episode(self, obs, optimal_actions):` generates episode rollout and stores in `self.replay_buffer`.
5. `load_overall_stats(self):`: loads model dumped stats from `stats/{model_name}`
6. `save_overall_stats(self):` saves model overall stats (trainiing) in `stats/{model_name}`
7. `save_algo_policy(self):` save algorithm policy state dict.
## A2C(PolicyGradientBase):
This class implement Advantage Actor Critc with support for learn by demonstration for a discrete action space. It also support batched updates specified by `update_every_n` to update every certain number of episodes stored in the replay buffer. 
It has two main attributes:
1. Policy (Actor) Optimizer: self.Poptim
2. Value (Critic) Optimizer: self.Voptim
### Functions
1. The policy $\pi$ (Actor) loss  is computed as the following:
```
    log_pi, G, v = replay_buffer.logps, replay_buffer.Gs, replay_buffer.vs
    advantage = G - v.detach() if not self.learn_by_demo else 1
    if self.clip_range:
        c_lo, c_hi = self.clip_range
        pi_comp = torch.min(log_pi, log_pi.clamp(min=c_lo, max=c_hi))
        loss_pi = self.alpha * (-pi_comp*advantage).mean()
    else:
        loss_pi = self.alpha * (-log_pi*advantage).mean()
    return loss_pi
```
2. The value $\hat{v}$ (Critic) loss  is computed as the following:
```
    G, v = replay_buffer.Gs, replay_buffer.vs
    loss_v= self.alpha*((G - v).pow(2)).mean()
    return loss_v
```
3. `update_agent(self):` perform a backward for both actor and critic's networks.
## SAC(PolicyGradientBase):
This class implements the Soft Actor Critic with support for learning by demonstration if expert trajectories were supplied.
It has two main attributes:
1. Policy (Actor) Optimizer: self.Poptim
2. Q-value (Critic) Optimizer: self.Qoptim
### functions:
1. The policy $\pi$ (Actor) loss  is computed as the following:
```
s, a =replay_buffer.states, replay_buffer.actions
q1_pi = self.network.q1(s,a)
q2_pi = self.network.q2(s,a)
q_pi = torch.min(q1_pi, q2_pi)

loss_pi = (-self.alpha * replay_buffer.logps - q_pi).mean()
```
2. The value $\hat{Q}$ (Critic) loss  is computed as the following:
```
    s, a, r, s_next,  d = replay_buffer.states, replay_buffer.actions, replay_buffer.rewards, replay_buffer.states_next, replay_buffer.donez
    # Current Qs
    q1 = self.network.q1(s,a)
    q2 = self.network.q2(s,a)
    with torch.no_grad():
        a2, (logp_a2, _) = self.network.pi.predict(s_next, deterministic=True)
        t_q1 = self.target_network.q1(s_next, a2)
        t_q2 = self.target_network.q2(s_next, a2)
        q_pi_targ = torch.min(t_q1, t_q2)
        bellman = r + self.gamma * (1 - d) * (q_pi_targ - self.alpha * logp_a2)

    # MSE loss against Bellman backup
    loss_q1 = ((q1 - bellman)**2).mean()
    loss_q2 = ((q2 - bellman)**2).mean()
    loss_q = loss_q1 + loss_q2
```
# Other directories
1. `stable_baselines_experiments/:` contains the main pytohn script for experiments done on agents from stable baselines project (ACER, A2C, PPO1). 




## Code with Final Results

In [6]:
from imitation.algorithms import bc
from algorithms.sac import SAC
from algorithms.a2c import A2C
from networks.mlpsac import MLPSAC
from networks.mlpa2c import MLPA2C
from stable_baselines3.common import policies
from utils import *
import torch

Hyperparameeters and Initializations

In [3]:
dataset_dict, dataset_all = load_datasets(dataset_dir="dataset")
hidden_acts_a2c = {
    "pi":nn.ReLU,
    "v": nn.ReLU
}
archs = { 
    'SAC':{
        "pi": [256, 256],
        "q" : [256, 256]
    }, 
    'SAC with Demo (512, 256)':{
        "pi": [512, 256],
        "q" : [512, 256]
    }, 
    'A2C':{
        "pi": [256, 256],
        "v" : [256, 256]
    },   
    'A2C with Demo (512, 256)':{
        "pi": [512, 256],
        "v" : [512, 256]
    }
}
hidden_acts_sac = {
    "pi":nn.ReLU,
    "q": nn.ReLU
}
hidden_acts_a2c = {
    "pi":nn.ReLU,
    "v": nn.ReLU
}
state_type = 'S0'
dataset_dict, dataset_all = load_datasets(dataset_dir="dataset")
expert_trajectories = dataset_to_trajs(dataset_dict, save_path="trajectories", traj_obj=False, state_space=state_type)
env = Karel(dataset_all['train_task'], state_type, sequential_loading=False)
state_size = env.observation_space.shape[0]
n_actions = env.action_space.n
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

 33%|███▎      | 1/3 [00:01<00:02,  1.02s/it]

Saved  trajectories/data_easy_S0_traj.pkl


 67%|██████▋   | 2/3 [00:03<00:02,  2.07s/it]

Saved  trajectories/data_medium_S0_traj.pkl


100%|██████████| 3/3 [00:09<00:00,  3.30s/it]

Saved  trajectories/data_S0_traj.pkl


Saved  trajectories/all_S0_traj.pkl


# Behavioral Cloning Experiment (best model: BC-FFNN)

In [7]:
arch = [128,128, 128]
class FeedForwardPolicy(policies.ActorCriticPolicy):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs, net_arch=arch)
best_model_path = "models/FeedForward_S0_[128, 128, 128]_15_32_0.89125_0.8791666666666667.zip"
policy = bc.reconstruct_policy(best_model_path)
stats = eval_model(policy, dataset_dict['data'], state_type="S0", show_every_n=1e10)
print(stats)

{'avg_reward': 0.20027979196728923, 'avg_extra_steps': 0.00925497454881999, 'solved_percentage': 0.9004166666666666, 'solved_optimally_percentage': 0.8975, 'avg_disc_returns': 0.8644595549614149}


# A2C Best Model: Learning with Demo

In [8]:
for experiment in ['A2C with Demo (512, 256)']:
    network = MLPA2C(state_size, n_actions, archs[experiment], hidden_acts_a2c, device)
    policy = A2C(env,network=network,model_name=experiment)
    policy.load_algo_policy()
    stats = eval_model(policy.network.pi, dataset_dict['data'], state_type="S0", show_every_n=1e10)
    print(stats)

{'avg_reward': 0.19575117243866955, 'avg_extra_steps': 0.01810385898046689, 'solved_percentage': 0.8745833333333334, 'solved_optimally_percentage': 0.8683333333333333, 'avg_disc_returns': 0.8399508973367106}


# SAC Best Model: Learning with Demo

In [9]:
for experiment in ['SAC with Demo (512, 256)']:
    network = MLPSAC(state_size, n_actions, archs[experiment], hidden_acts_sac, device)
    policy = SAC(env,network=network,model_name=experiment)
    policy.load_algo_policy()
    stats = eval_model(policy.network.pi, dataset_dict['data'], state_type="S0", show_every_n=1e10)
    print(stats)

{'avg_reward': 0.19601603835978543, 'avg_extra_steps': 0.007655502392344498, 'solved_percentage': 0.8708333333333333, 'solved_optimally_percentage': 0.8679166666666667, 'avg_disc_returns': 0.8373242925687909}
